# **NLP4_Run1**

In [1]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 74.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 8.2 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [2]:

import openai
import os
from IPython.display import Markdown
import pyomo


### **Accessing the GPT4 API**

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']



### **Generate Mathematical Model**

In [ ]:
problem = """ We are looking at an alkylation process which will include the following 10 variables: olefin feed (barrels per day), isobutane recycle (barrels per day), acid addition rate (thousands of pounds per day), alkylate yield (barrels per day), isobutane makeup (barrels per day), acid strength (weight per cent), motor octane number, external isobutane-to-olefin ratio, acid dilution factor and F-4 performance number.

We want to maximize the daily profit of this alkylation process.
The profit is defined as the revenue generated from the alkylate yield multiplied with the motor octane number, minus the operational costs, which include olefin feed, isobutane recycle, acid addition rate, and isobutane makeup.

Relationships in terms of other variables for alkylate yield, motor octane number, acid dilution factor, and F-4 performance number can be formulated as regression formulas.
This regression estimate can deviate in both directions from true value of these variables by 2, 1, 5 and 10 percent, respectively.
Alkylate yield is a function of olefin feed and external isobutane-to-olefine yield. Alkalyte yield equals the amount of olefin feed multiplied by the sum of 1.12, 0.13167 times the external isobutane-to-olefin ratio and -0.00667 times the external isobutane-to-olefin ratio squared.
The motor octane number is derived from the external isobutane-to-olefin ratio and the acid strength. The motor octane number is calculated as the sum of 86.35, 1.098 time external isobutane-to-olefin ratio, -0.038 times the external isobutane-to-olefin ratio squared and 0.325 times acid strength reduced by 89.
The acid dilution factor is calculated based on the F-4 performance number. The acid dillution factor is expressed as 35.82 minus 0.222 times F-4 performance number.
Lastly, the F-4 performance number depends on the motor octane number. F-4 performance number is calculated as -133 plus three times the motor octane number.

There are some additional constraints imposed by the nature of the chemical process.
Each variable has a lower and an upper bound.
The external isobutane-to-olefin ratio needs to equal the ratio of isobutane recycle plus isobutane makeup to olefin feed.
The acid strength needs to equal the ratio of 93000 times acid addition rate to acid addition rate multiplied by acid dilution factor in addition to 1000 times acid addition rate.
Lastly, 1.22 alkylate yield needs to be equal to the combined olefin feed and isobutane makeup. """

In [ ]:
client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 1
)


In [ ]:
print(response1.choices[0].message.content)

Mathematical Optimization Model:

Parameters:

- Price of alkylate
- Costs of olefin, acid, isobutane recycle, and isobutane makeup

Decision variables:

- X1: Olefin feed (barrels/day)
- X2: Isobutane recycle (barrels/day)
- X3: Acid addition rate (thousands of pounds/day)
- X4: Alkylate yield (barrels/day)
- X5: Isobutane makeup (barrels/day)
- X6: Acid strength (weight per cent)
- X7: Motor octane number
- X8: External isobutane-to-olefin ratio
- X9: Acid dilution factor
- X10: F-4 performance number 

Objective function: 

Max Profit= Price of alkylate * X4 * X7 - Cost of olefin * X1 - Cost of isobutane recycle * X2 - Cost of acid * X3 - Cost of isobutane makeup * X5 

Constraints:

Regression formulas: 

- X4 = X1 * (1.12 + 0.13167 * X8 - 0.00667 * X8^2) +/- 0.02 * X4
- X7 = 86.35 + 1.098 * X8 - 0.038 * X8^2 + 0.325 * (X6 - 89) +/- 0.01 * X7
- X9 = 35.82 - 0.222 * X10 +/- 0.05 * X9
- X10 = -133 + 3 * X7 +/- 0.10 * X10

Chemical process constraints: 

- X1_min <= X1 <= X1_max
- X2_

### **Generate Pyomo Code**

In [ ]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 1
)

In [ ]:
Markdown(response2.choices[0].message.content)

Here is the sample Python code using the Pyomo package:

```python
from pyomo.environ import *

# Creating a concrete model
model = ConcreteModel()

# Parameters (Sample Data)
Price_alkylate = 200
Cost_olefin = 100
Cost_isobu_recycle = 50
Cost_acid = 80
Cost_isobu_makeup = 30

# Variables
model.X1 = Var(domain=NonNegativeReals)
model.X2 = Var(domain=NonNegativeReals)
model.X3 = Var(domain=NonNegativeReals)
model.X4 = Var(domain=NonNegativeReals)
model.X5 = Var(domain=NonNegativeReals)
model.X6 = Var(domain=NonNegativeReals)
model.X7 = Var(domain=NonNegativeReals)
model.X8 = Var(domain=NonNegativeReals)
model.X9 = Var(domain=NonNegativeReals)
model.X10 = Var(domain=NonNegativeReals)

# Objective function
def profit(model):
    return Price_alkylate * model.X4 * model.X7 - Cost_olefin * model.X1 - Cost_isobu_recycle * model.X2 - \
           Cost_acid * model.X3 - Cost_isobu_makeup * model.X5
model.profit = Objective(rule=profit, sense=maximize)

# Constraints
model.cons1 = Constraint(expr=model.X4 == model.X1 * (1.12 + 0.13167 * model.X8 - 0.00667 * model.X8 ** 2))
model.cons2 = Constraint(expr=model.X7 == 86.35 + 1.098 * model.X8 - 0.038 * model.X8 ** 2 + 0.325 * 
                           (model.X6 - 89))
model.cons3 = Constraint(expr=model.X9 == 35.82 - 0.222 * model.X10)
model.cons4 = Constraint(expr=model.X10 == -133 + 3 * model.X7)
model.cons5 = Constraint(expr=model.X8 == (model.X2 + model.X5) / model.X1)
model.cons6 = Constraint(expr=model.X6 == 93000 * model.X3 / (model.X3 * model.X9 + 1000 * model.X3))
model.cons7 = Constraint(expr=1.22 * model.X4 == model.X1 + model.X5)

from pyomo.opt import SolverFactory
opt = SolverFactory('ipopt')
opt.solve(model)

print("Profit = ", model.profit())
print("X1 = ", model.X1())
print("X2 = ", model.X2())
print("X3 = ", model.X3())
print("X4 = ", model.X4())
print("X5 = ", model.X5())
print("X6 = ", model.X6())
print("X7 = ", model.X7())
print("X8 = ", model.X8())
print("X9 = ", model.X9())
print("X10 = ", model.X10())

```

Please note, you need to install 'ipopt' solver and allocate appropriate bounds for decision variables. You also have to modify the problem based on the specific requirement.

### **Run the code Generated by GPT4**

In [3]:
# Install Python API for AMPL
!pip install amplpy --upgrade

from amplpy import AMPL,ampl_notebook

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 37.1 MB/s eta 0:00:00


In [ ]:

_ = load_dotenv(find_dotenv('second_knitro_license.env'))
my_knitro_license = os.environ['knitro_license']

ampl = ampl_notebook(
    modules=["knitro"],  # modules to install
    license_uuid=my_knitro_license,  # license to use
)

Licensed to AMPL Community Edition License for <cevikmelis14@gmail.com>.


In [ ]:
from pyomo.environ import *

# Creating a concrete model
model = ConcreteModel()

# Parameters (Sample Data)
Price_alkylate = 200
Cost_olefin = 100
Cost_isobu_recycle = 50
Cost_acid = 80
Cost_isobu_makeup = 30

# Variables
model.X1 = Var(domain=NonNegativeReals)
model.X2 = Var(domain=NonNegativeReals)
model.X3 = Var(domain=NonNegativeReals)
model.X4 = Var(domain=NonNegativeReals)
model.X5 = Var(domain=NonNegativeReals)
model.X6 = Var(domain=NonNegativeReals)
model.X7 = Var(domain=NonNegativeReals)
model.X8 = Var(domain=NonNegativeReals)
model.X9 = Var(domain=NonNegativeReals)
model.X10 = Var(domain=NonNegativeReals)

# Objective function
def profit(model):
    return Price_alkylate * model.X4 * model.X7 - Cost_olefin * model.X1 - Cost_isobu_recycle * model.X2 - \
           Cost_acid * model.X3 - Cost_isobu_makeup * model.X5
model.profit = Objective(rule=profit, sense=maximize)

# Constraints
model.cons1 = Constraint(expr=model.X4 == model.X1 * (1.12 + 0.13167 * model.X8 - 0.00667 * model.X8 ** 2))
model.cons2 = Constraint(expr=model.X7 == 86.35 + 1.098 * model.X8 - 0.038 * model.X8 ** 2 + 0.325 *
                           (model.X6 - 89))
model.cons3 = Constraint(expr=model.X9 == 35.82 - 0.222 * model.X10)
model.cons4 = Constraint(expr=model.X10 == -133 + 3 * model.X7)
model.cons5 = Constraint(expr=model.X8 == (model.X2 + model.X5) / model.X1)
model.cons6 = Constraint(expr=model.X6 == 93000 * model.X3 / (model.X3 * model.X9 + 1000 * model.X3))
model.cons7 = Constraint(expr=1.22 * model.X4 == model.X1 + model.X5)

from pyomo.opt import SolverFactory
opt = SolverFactory('knitro')
opt.solve(model)

print("Profit = ", model.profit())
print("X1 = ", model.X1())
print("X2 = ", model.X2())
print("X3 = ", model.X3())
print("X4 = ", model.X4())
print("X5 = ", model.X5())
print("X6 = ", model.X6())
print("X7 = ", model.X7())
print("X8 = ", model.X8())
print("X9 = ", model.X9())
print("X10 = ", model.X10())

  - termination condition: optimal
  - message from solver: Knitro 14.0.0\x3a Relative change in feasible solution estimate < xtol for xtol_iters.; objective 234598961840176700; feasibility error 9.48e-09; 114 iterations; 153 function evaluations


Profit =  2.345989618401767e+17
X1 =  11320313090548.553
X2 =  632428637396.1982
X3 =  6.49766837442244e-11
X4 =  13406066912491.611
X5 =  5035088542691.214
X6 =  92.37680533599034
X7 =  87.98765095583724
X8 =  0.500650224251293
X9 =  6.746224463413831
X10 =  130.9629528675114


In [ ]:
from pyomo.environ import *

# Creating a concrete model
model = ConcreteModel()

# data inputted by human
Price_alkylate = 0.63
Cost_olefin = 5.04
Cost_isobu_recycle = 0.035
Cost_acid = 10.0
Cost_isobu_makeup =  3.36
#end
# Variables
model.X1 = Var(domain=NonNegativeReals)
model.X2 = Var(domain=NonNegativeReals)
model.X3 = Var(domain=NonNegativeReals)
model.X4 = Var(domain=NonNegativeReals)
model.X5 = Var(domain=NonNegativeReals)
model.X6 = Var(domain=NonNegativeReals)
model.X7 = Var(domain=NonNegativeReals)
model.X8 = Var(domain=NonNegativeReals)
model.X9 = Var(domain=NonNegativeReals)
model.X10 = Var(domain=NonNegativeReals)

# Objective function
def profit(model):
    return Price_alkylate * model.X4 * model.X7 - Cost_olefin * model.X1 - Cost_isobu_recycle * model.X2 - \
           Cost_acid * model.X3 - Cost_isobu_makeup * model.X5
model.profit = Objective(rule=profit, sense=maximize)

# Constraints
model.cons1 = Constraint(expr=model.X4 == model.X1 * (1.12 + 0.13167 * model.X8 - 0.00667 * model.X8 ** 2))
model.cons2 = Constraint(expr=model.X7 == 86.35 + 1.098 * model.X8 - 0.038 * model.X8 ** 2 + 0.325 *
                           (model.X6 - 89))
model.cons3 = Constraint(expr=model.X9 == 35.82 - 0.222 * model.X10)
model.cons4 = Constraint(expr=model.X10 == -133 + 3 * model.X7)
model.cons5 = Constraint(expr=model.X8 == (model.X2 + model.X5) / model.X1)
model.cons6 = Constraint(expr=model.X6 == 93000 * model.X3 / (model.X3 * model.X9 + 1000 * model.X3))
model.cons7 = Constraint(expr=1.22 * model.X4 == model.X1 + model.X5)

from pyomo.opt import SolverFactory
opt = SolverFactory('knitro')
opt.solve(model)

print("Profit = ", model.profit())
print("X1 = ", model.X1())
print("X2 = ", model.X2())
print("X3 = ", model.X3())
print("X4 = ", model.X4())
print("X5 = ", model.X5())
print("X6 = ", model.X6())
print("X7 = ", model.X7())
print("X8 = ", model.X8())
print("X9 = ", model.X9())
print("X10 = ", model.X10())

  - termination condition: infeasible
  - message from solver: Knitro 14.0.0\x3a Convergence to an infeasible point. Problem may be locally infeasible.; objective 470424440202.9827; feasibility error 0.572; 136 iterations; 195 function evaluations


Profit =  470424440202.9826
X1 =  7333848688.411778
X2 =  712647152.0726184
X3 =  73558.12115352809
X4 =  9318160041.225014
X5 =  4034306561.8827376
X6 =  92.42306600171572
X7 =  88.74427003287272
X8 =  1.2187792843194962
X9 =  6.242316158106762
X10 =  133.2328100986182


### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**

In [5]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('second_knitro_license.env'))
my_knitro_license = os.environ['knitro_license']

ampl = ampl_notebook(
    modules=["coin"],  # modules to install
    license_uuid=my_knitro_license,  # license to use
)

Licensed to AMPL Community Edition License for <cevikmelis14@gmail.com>.


In [7]:
from pyomo.environ import *

# Creating a concrete model
model = ConcreteModel()

# data inputted by human
Price_alkylate = 0.63
Cost_olefin = 5.04
Cost_isobu_recycle = 0.035
Cost_acid = 10.0
Cost_isobu_makeup =  3.36

# Variables
model.X1 = Var(domain=NonNegativeReals, bounds=(0, 2000))
model.X2 = Var(domain=NonNegativeReals, bounds =(0, 16000))
model.X3 = Var(domain=NonNegativeReals, bounds=(0, 120))
model.X4 = Var(domain=NonNegativeReals, bounds=(0, 5000))
model.X5 = Var(domain=NonNegativeReals, bounds=(0, 2000))
model.X6 = Var(domain=NonNegativeReals, bounds=(85, 93))
model.X7 = Var(domain=NonNegativeReals,bounds=(90, 95))
model.X8 = Var(domain=NonNegativeReals, bounds=(3, 12))
model.X9 = Var(domain=NonNegativeReals,bounds=(1.2, 4))
model.X10 = Var(domain=NonNegativeReals,bounds=(145, 162))
#end

# Objective function
def profit(model):
    return Price_alkylate * model.X4 * model.X7 - Cost_olefin * model.X1 - Cost_isobu_recycle * model.X2 - \
           Cost_acid * model.X3 - Cost_isobu_makeup * model.X5
model.profit = Objective(rule=profit, sense=maximize)

# Constraints
model.cons1 = Constraint(expr=model.X4 == model.X1 * (1.12 + 0.13167 * model.X8 - 0.00667 * model.X8 ** 2) + (4000*model.X4))
model.cons11 = Constraint(expr=model.X4 == model.X1 * (1.12 + 0.13167 * model.X8 - 0.00667 * model.X8 ** 2) - (0*model.X4))

model.cons2 = Constraint(expr=model.X7 == 86.35 + 1.098 * model.X8 - 0.038 * model.X8 ** 2 + 0.325 *
                           (model.X6 - 89) + (95*model.X7))

model.cons22 = Constraint(expr=model.X7 == 86.35 + 1.098 * model.X8 - 0.038 * model.X8 ** 2 + 0.325 *
                           (model.X6 - 89) - (90*model.X7))

model.cons3 = Constraint(expr=model.X9 == 35.82 - 0.222 * model.X10 +(4*model.X9))
model.cons33 = Constraint(expr=model.X9 == 35.82 - 0.222 * model.X10 - (1.2*model.X9))

model.cons4 = Constraint(expr=model.X10 == -133 + 3 * model.X7 +(162*model.X10))
model.cons44 = Constraint(expr=model.X10 == -133 + 3 * model.X7 - (145*model.X10))

model.cons5 = Constraint(expr=model.X8 == (model.X2 + model.X5) / model.X1)
model.cons6 = Constraint(expr=model.X6 == 93000 * model.X3 / (model.X3 * model.X9 + 1000 * model.X3))
model.cons7 = Constraint(expr=1.22 * model.X4 == model.X1 + model.X5)


from pyomo.opt import SolverFactory
opt = SolverFactory('couenne')
opt.solve(model)



  - termination condition: infeasible
  - message from solver: Couenne (/tmp/tmpp0kl11tr.pyomo.nl Feb 21 2023)\x3a Infeasible


{'Problem': [{'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 0, 'Number of variables': 0, 'Sense': 'unknown'}], 'Solver': [{'Status': 'warning', 'Message': 'Couenne (/tmp/tmpp0kl11tr.pyomo.nl Feb 21 2023)\\x3a Infeasible', 'Termination condition': 'infeasible', 'Id': 220, 'Error rc': 0, 'Time': 0.046578168869018555}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}